<a href="https://colab.research.google.com/github/mattmason36/pyTorch-testing/blob/main/simple_neuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IRIS Dataset
https://archive.ics.uci.edu/dataset/53/iris

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [10]:
# Create Model class that inherits nn.Module
class Model(nn.Module):
  # Input layer (4 features of the flower) -->
  # Hidden layer1 (number of neurons)  -->
  # H2 (n)  -->
  # output(3 classes of iris flowers)
  def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
    super().__init__() # instantiate our nn.Module
    self.fc1 = nn.Linear(in_features, h1)
    self.fc2 = nn.Linear(h1, h2)
    self.out = nn.Linear(h2, out_features)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x

In [11]:
# Pick a manual seed for randomisation
torch.manual_seed(41)
#Create an instance of model
model = Model()